In [117]:
import os
M = []
UWL = []
def scan(path):
    try:
        for i in os.scandir(path):
            if (i.is_file()):
                name = i.name
                if name.endswith(".txt"):
                    file = open(i.path)
                    txt = file.read()
                    wl = txt.split(" ")
                    row = [0 for w in UWL]
                    for w in wl:
                        if w not in UWL:
                            for ind, prevRow in enumerate(M):
                                M[ind] = prevRow + [0]
                            UWL.append(w)
                            row.append(0)
                        row[UWL.index(w)] = wl.count(w)
                    M.append(row)
            else:
                scan(i)
    except:
        pass

scan("C:\\src\\flutter")

In [118]:
file = open("vsm.txt", "w")
for row in M:
    strRow = str(row)
    strRow = strRow[1:]
    strRow = strRow[:-1]
    file.write(strRow+"\n")
file.close()

In [78]:
file = open("C:\\src\\flutter\\examples\\api\windows\\CMakeLists.txt")

NameError: name 'txt' is not defined

In [116]:
str([1, 2, 3])

'[1, 2, 3]'

In [82]:
M = [[0,0,1,1,1],
    [1,1,0,0,0],
    [1,0,0,0,0],
    [1,1,1,1,0]]
r= []
for i in range(len(M)):
    count = 0
    for j in range(len(M[0])):
        if M[i][j] == 1:
            count += 1
    r.append(count)
    m = r[0]
    for k in range(len(r)):
        if r[k] > m:
            m = r[k]
print(f"The maximum no of 1s occur at {k+1}.")

The maximum no of 1s occur at 4.


In [86]:
maxCount = 0
maxInd = 0
for ind, row in enumerate(M):
    maxRow = row.count(1)
    if maxRow > maxCount:
        maxCount = maxRow
        maxInd = ind
print(maxInd)

4
